# Machine Offsets from Least Squares

This notebook will call an arbor probing part program, then solve for the best fit machine offsets using the probing data.

You will need the ruby probe mounted in the collet, and an empty arbor clamped in the spindle.

## Probing part program

Here we will call a part program to probe the arbor in the spindle.

    Note: to run the cell use SHIFT + ENTER

In [ ]:
#Run the probing part program synchronously
import os.path
from amcnc import program

status = program.run(os.path.abspath(r'probe_arbor.pp'))

## Solver

Now that we have a set of probing data, we can use this to solve for the best fit machine offsets.

First we will read the output text file and visualise the data with a 3D plot.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import exp, loadtxt, pi, sqrt, sin, cos
import scipy
from scipy.stats import norm

In [ ]:
#read the measured probing data
arbor_data = loadtxt('P:/TG7/misc/arbor_probing_data.txt')

x = arbor_data[:, 0]
y = arbor_data[:, 1]
z = arbor_data[:, 2]
a = arbor_data[:, 3]
c = arbor_data[:, 4]

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

Axes3D = Axes3D  # pycharm auto import
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

ax.scatter3D(x, y, z)

ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_zlabel('Z (mm)')

Now we will define a function **arbor_and_spindle_face** which models the machine including the parameters of intrest, then we use **scipy.optimize.leastsq** to solve.

In [ ]:
def arbor_and_spindle_face(vars, data, num_arbor_pts, ball_radius):
    """This function uses data from the arbor as well as data from the spindle front face 
    to solve for all the machine offsets"""
    
    ball_runout = vars[0]
    ball_runout_angle = vars[1]
    arbor_radius = vars[2]
    spindle_centerline_offset = vars[3]
    x_offset = vars[4]
    y_home_offset = vars[5]
    z_home_offset = vars[6]
    c_home_offset = vars[7]
    spindle_ref_position= vars[8]
    
    x = data[:, 0]
    y = data[:, 1]
    z = data[:, 2]
    a = data[:, 3]
    c = data[:, 4]
    
    error = np.zeros(data.shape[0])

    ball_runout_angle_radians = (ball_runout_angle/360.0)*2*pi
    c_home_offset_radians = (c_home_offset/360.0)*2*pi
    
    for i in range(0, data.shape[0]):

        a_angle_radians = (a[i]/360.0)*2*pi
        
        ball_x = x[i] + x_offset
        ball_y = y[i] + y_home_offset + ball_runout*sin(a_angle_radians + ball_runout_angle_radians)
        ball_z = z[i] + z_home_offset + ball_runout*cos(a_angle_radians + ball_runout_angle_radians)

        c_angle_radians = (c[i]/360.0)*2*pi

        #find equation of the c-axis vector
        ui = -sin(c_angle_radians + c_home_offset_radians)
        uj = cos(c_angle_radians + c_home_offset_radians)
        uk = 0

        #coords of the spindle axis origin
        x0 = cos(c_angle_radians + c_home_offset_radians)*(-spindle_centerline_offset)
        y0 = sin(c_angle_radians + c_home_offset_radians)*(-spindle_centerline_offset)
        z0 = 0
        
        #find the closest point on the spindle axis
        point_x = x0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*ui;
        point_y = y0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*uj;
        point_z = z0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*uk;
        
        if i < num_arbor_pts:
            #the error for each arbor point is the distance to the closest point on the spindle axis vector, minus the ball and arbor radius
            error[i] = np.sqrt(((point_x - ball_x)**2 + (point_y - ball_y)**2 + (point_z - ball_z)**2)) - (ball_radius + arbor_radius)

        else:
            #the error for each spindle face point: the distance from the closest point on the spindle axis vector to the spindle axis origin
            error[i] = np.sqrt(((point_x - x0)**2 + (point_y - y0)**2 + (point_z - z0)**2)) - (-spindle_ref_position - ball_radius)

        
    return (error)

In [ ]:
#read the operator entered probe diameter
#Read and write IPF
ipf110 = variable.CncFloat('IPF110')
ball_diameter = ipf110.value
print('operator entered ball_diameter = ', ball_diameter)


In [ ]:
vars = np.zeros(9)

vars[0] = 0.0 #ball_runout
vars[1] = 0.0 #ball_runout_angle
vars[2] = 0.0 #radius
vars[3] = 0.0 #spindle_centerline_offset
vars[4] = 0.0 #x_offset
vars[5] = 0.0 #y_home_offset
vars[6] = 0.0 #z_home_offset
vars[7] = 0.0 #c_home_offset
vars[8] = 0.0 #spindle_ref_position

ball_radius = ball_diameter/2 #we need the probe ball radius now, so we can determine the spindle referance position

p, success, infodict,mesg,ier  = scipy.optimize.leastsq(arbor_and_spindle_face, vars, args=(arbor_data, arbor_data.shape[0], ball_radius), full_output=1)

print(mesg)
print("\nsolver converged in ", infodict['nfev'], " iterations\n")

p_three_decimals = ["%.3f" % v for v in p]

print('Results:')
print('ball_runout \t\t\t=\t', p_three_decimals[0], "mm")
print('ball_runout_angle \t\t=\t', p_three_decimals[1], "deg")
print('combined_radius \t\t=\t', p_three_decimals[2], "mm")
print('spindle_centerline_offset \t=\t', p_three_decimals[3], "mm")
print('x_offset \t\t\t=\t', p_three_decimals[4], "mm")
print('y_home_offset \t\t\t=\t', p_three_decimals[5], "mm")
print('z_home_offset \t\t\t=\t', p_three_decimals[6], "mm")
print('c_home_offset \t\t\t=\t', p_three_decimals[7], "deg")

## Results

Here we do some analysis on the result by plotting a histogram. 

This histogram and standard deviaton give an idea of how well the solution conforms to the data.

In [ ]:
%matplotlib notebook

# best fit of data
(mu, sigma) = norm.fit(infodict['fvec'])

# the histogram of the data
n, bins, patches = plt.hist(infodict['fvec'], 15, density=True)

# add a 'best fit' line
y = scipy.stats.norm.pdf( bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

plt.xlabel('Residual error (mm)')
plt.ylabel('Number of points (normalised)')
plt.title(r'$\mathrm{Histogram\ of\ Residual\ Errors:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))

plt.show()

We can also try and visualise how the error is distributed with a 3D plot.

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

Axes3D = Axes3D  # pycharm auto import
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

# Normalised [0,1]
fvec_normalised = (infodict['fvec'] - np.min(infodict['fvec']))/np.ptp(infodict['fvec'])

ax.scatter3D(arbor_data[:,0], arbor_data[:,1], arbor_data[:,2], s = 20000*np.abs(infodict['fvec']), c = fvec_normalised, cmap = 'cool')

ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_zlabel('Z (mm)')

Here the size of the point relates to the magnatude of the residual error, and the colour indicates positive or negative deviation of the radius measured at each point.

Now run a part program to probe the spindle front face. 

## First we need to remove the arbor!

In [ ]:
from amcnc import program

status = program.run(os.path.abspath(r'probe_spindle.pp'))

In [ ]:
#read the measured spindle face probing data

spindle_data = loadtxt('P:/TG7/misc/spindle_probing_data.txt')
print("there are", spindle_data.shape[0], " spindle probe points")

num_arbor_pts = arbor_data.shape[0]

print("there are", num_arbor_pts, " arbor probe points")

#concatinate the spindle data and the arbor data, creating the full data set

full_data = np.concatenate((arbor_data, spindle_data), axis=0)

print("there are", full_data.shape[0], " probe points total")

Now that we have a set of spindle face probing data we can include the **spindle_ref_position** in the solution.

In [ ]:
vars = np.zeros(9)

vars[0] = 0.0 #ball_runout
vars[1] = 0.0 #ball_runout_angle
vars[2] = 0.0 #arbor_radius
vars[3] = 0.0 #spindle_centerline_offset
vars[4] = 0.0 #x_offset
vars[5] = 0.0 #y_home_offset
vars[6] = 0.0 #z_home_offset
vars[7] = 0.0 #c_home_offset
vars[8] = 0.0 #spindle_ref_position

ball_radius = ball_diameter/2 #we need the probe ball radius now, so we can determine the spindle referance position

p, success, infodict,mesg,ier  = scipy.optimize.leastsq(arbor_and_spindle_face, vars, args=(full_data, num_arbor_pts, ball_radius), full_output=1)

print(mesg)
print("\nsolver converged in ", infodict['nfev'], " iterations\n")

p_three_decimals = ["%.3f" % v for v in p]

print('Results:')
print('ball_runout \t\t\t=\t', p_three_decimals[0], "mm")
print('ball_runout_angle \t\t=\t', p_three_decimals[1], "deg")
print('arbor_radius \t\t\t=\t', p_three_decimals[2], "mm")
print('spindle_centerline_offset \t=\t', p_three_decimals[3], "mm")
print('x_offset \t\t\t=\t', p_three_decimals[4], "mm")
print('y_home_offset \t\t\t=\t', p_three_decimals[5], "mm")
print('z_home_offset \t\t\t=\t', p_three_decimals[6], "mm")
print('c_home_offset \t\t\t=\t', p_three_decimals[7], "deg")
print('spindle_ref_position \t\t=\t', p_three_decimals[8], "mm")


In [ ]:
%matplotlib notebook

# best fit of data
(mu, sigma) = norm.fit(infodict['fvec'])

# the histogram of the data
n, bins, patches = plt.hist(infodict['fvec'], 30, density=True)

# add a 'best fit' line
y = scipy.stats.norm.pdf( bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

plt.xlabel('Residual error (mm)')
plt.ylabel('Number of points (normalised)')
plt.title(r'$\mathrm{Histogram\ of\ Residual\ Errors:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))

plt.show()

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

Axes3D = Axes3D  # pycharm auto import
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

# Normalised [0,1]
fvec_normalised = (infodict['fvec'] - np.min(infodict['fvec']))/np.ptp(infodict['fvec'])

ax.scatter3D(full_data[:,0], full_data[:,1], full_data[:,2], s = 20000*np.abs(infodict['fvec']), c = fvec_normalised, cmap = 'cool')

ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_zlabel('Z (mm)')

## Save New Offsets to the Machine

Now we can save the results to the machine.

First copy the result to machine variables.

In [ ]:
from amcnc import variable
ipf100 = variable.CncFloat('IPF100')
ipf101 = variable.CncFloat('IPF101')
ipf102 = variable.CncFloat('IPF102')
ipf103 = variable.CncFloat('IPF103')
ipf104 = variable.CncFloat('IPF104')
ipf105 = variable.CncFloat('IPF105')
ipf106 = variable.CncFloat('IPF106')
ipf107 = variable.CncFloat('IPF107')
ipf108 = variable.CncFloat('IPF108')
ipf109 = variable.CncFloat('IPF109')

ipf100.value = p[0] #ball_runout
ipf101.value = p[1] #ball_runout_angle
ipf102.value = p[2] #arbor_radius
ipf103.value = p[3] #spindle_centerline_offset
ipf104.value = p[4] #x_offset
ipf105.value = p[5] #y_home_offset
ipf106.value = p[6] #z_home_offset
ipf107.value = p[7] #c_home_offset
ipf108.value = p[8] #spindle_ref_position


Now run a part program to review changes and save the new offsets to the database

In [ ]:
from amcnc import program

status = program.run(os.path.abspath(r'write_results.pp'))